# Sentiment Analyzer

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
imdb, info = tfds.load('imdb_reviews',
                       with_info=True,
                       as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQ429A6/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQ429A6/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQ429A6/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train, test = imdb['train'],imdb['test']

In [ ]:
#info

In [ ]:
type(train)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [ ]:
train_sent = []
test_sent = []
y_train = []
y_test = []

In [ ]:
for sent, labels in train:
    train_sent.append(sent.numpy().decode('utf8'))
    y_train.append(labels)

In [ ]:
for sent, labels in test:
    test_sent.append(sent.numpy().decode('utf8'))
    y_test.append(labels)

In [ ]:
len(train_sent), len(test_sent), len(y_train), len(y_test)

(25000, 25000, 25000, 25000)

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
train_sent[0:2]

["This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
 'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development wa

In [ ]:
y_train[0:2]

array([0, 0])

In [ ]:
np.unique(y_train), np.unique(y_test)

(array([0, 1]), array([0, 1]))

In [ ]:
np.random.seed(12345)
tf.random.set_seed(12345)

In [ ]:
# to encode text to int
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
VOCAB = 1000 # Limit the vocabulary to 1000 words
EMBED_DIM = 32 # n-dimension for embedding layer
MAXLEN = 100 # Maximum length of Sentence

In [ ]:
token = Tokenizer(lower=True,
                  num_words=VOCAB,
                  oov_token="<OOV>")

In [ ]:
token.fit_on_texts(train_sent)

In [ ]:
train_sent_s = token.texts_to_sequences(train_sent)
test_sent_s = token.texts_to_sequences(test_sent)

In [ ]:
train_sent_s = pad_sequences(train_sent_s,
                             maxlen=MAXLEN,
                             padding='post',
                             truncating='post')

In [ ]:
test_sent_s = pad_sequences(test_sent_s,
                            maxlen=MAXLEN,
                            padding='post',
                            truncating='post')

In [ ]:
len(train_sent), len(test_sent), len(y_train), len(y_test)

(25000, 25000, 25000, 25000)

# Embeddings

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D

In [ ]:
emb_model = Sequential()

In [ ]:
emb_model.add(Embedding(VOCAB,
                        EMBED_DIM,
                        input_length = MAXLEN)) # Embedding layer

In [ ]:
emb_model.add(GlobalAveragePooling1D()) # Average Pooling layer

In [ ]:
emb_model.add(Dense(128, activation = 'relu')) # Dense layer - intermediate FC layer

In [ ]:
emb_model.add(Dense(1, activation='sigmoid')) # Output layer

In [ ]:
emb_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 32)           32000     
                                                                 
 global_average_pooling1d (  (None, 32)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 128)               4224      
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 36353 (142.00 KB)
Trainable params: 36353 (142.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
(128+1)*1

129

In [ ]:
emb_model.compile(optimizer = 'adam',
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy'])

In [ ]:
# Number of words = 1000
# Each word is represented by 32 numbers/features
# Total number of representations that need to be learnt by the model = 32*1000 = 32000 -> Number of parameters for the Embedding layer

In [ ]:
# word1 = [10, 2]
# word2 = [3,3]
# word3 = [4,7]
# sent1 = word1 word2 word3
# [3,2]
# Global Average Pooling - > [(10+3+4)/3, [(2+3+7)/3]

In [ ]:
(32+1)*128 # Remember the bias

4224

In [ ]:
len(y_test)

25000

In [ ]:
result = emb_model.fit(train_sent_s,
                       y_train,
                       validation_data=(test_sent_s,y_test),
                       epochs = 10)

Epoch 1/10
782/782 [==============================] - 25s 25ms/step - loss: 0.5129 - accuracy: 0.7399 - val_loss: 0.4328 - val_accuracy: 0.7964
Epoch 2/10
782/782 [==============================] - 8s 10ms/step - loss: 0.4218 - accuracy: 0.8028 - val_loss: 0.4293 - val_accuracy: 0.7998
Epoch 3/10
782/782 [==============================] - 7s 9ms/step - loss: 0.4151 - accuracy: 0.8081 - val_loss: 0.4376 - val_accuracy: 0.7910
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.4124 - accuracy: 0.8099 - val_loss: 0.4303 - val_accuracy: 0.7985
Epoch 5/10
782/782 [==============================] - 6s 8ms/step - loss: 0.4110 - accuracy: 0.8130 - val_loss: 0.4366 - val_accuracy: 0.7964
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.4097 - accuracy: 0.8113 - val_loss: 0.4303 - val_accuracy: 0.7984
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.4090 - accuracy: 0.8108 - val_loss: 0.4309 - val_accuracy: 0.7970
Epo

In [ ]:
new_reviews = ["I bought this for my husband who plays the piano.  He is having a wonderful time playing these old hymns.  The music  is at times hard to read because we think the book was published for singing from more than playing from.  Great purchase though!"]

In [ ]:
#new_reviews = ['This is a horribly bad product. Stopped working after few days. No response from customer service']

In [ ]:
new_reviews = token.texts_to_sequences(new_reviews)

In [ ]:
new_reviews = pad_sequences(new_reviews,
                            maxlen=MAXLEN,
                            padding='post',
                            truncating='post')

In [ ]:
new_reviews

array([[ 11,   1,  12,  16,  59, 657,  35, 297,   2,   1,  27,   7, 258,
          4, 387,  56, 394, 132, 152,   1,   2, 226,   7,  31, 209, 252,
          6, 330,  86,  73, 102,   2, 272,  14,   1,  16,   1,  37,  51,
         72, 394,  37,  85,   1, 149,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=int32)

In [ ]:
emb_model.predict(new_reviews)

1/1 [==============================] - 0s 19ms/step


array([[0.9703535]], dtype=float32)

In [ ]:
e = emb_model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(1000, 32)


In [ ]:
reverse_word_index = token.index_word

In [ ]:
#reverse_word_index

In [ ]:
import io
vectors = io.open('vectors_emb_22Oct2023.tsv', 'w', encoding='utf-8') # Vectors
metadata = io.open('metadata_emb_22Oct2023.tsv', 'w', encoding='utf-8') # Metadata
for word_num in range(1, VOCAB):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    metadata.write(word + "\n")
    vectors.write('\t'.join([str(x) for x in embeddings]) + "\n")
vectors.close()
metadata.close()

# Bidirectional LSTM

In [ ]:
# They said Teddy bears are on sale
# They said Teddy was a great president

In [ ]:
from tensorflow.keras.layers import  LSTM, Bidirectional

In [ ]:
bidi_model = Sequential()

In [ ]:
bidi_model.add(Embedding(VOCAB,
                         EMBED_DIM,
                         input_length = MAXLEN))

In [ ]:
bidi_model.add(Bidirectional(LSTM(128)))

In [ ]:
bidi_model.add(Dense(128,
                     activation = 'relu',
                     kernel_regularizer=tf.keras.regularizers.L2()))

In [ ]:
bidi_model.add(Dense(1, activation='sigmoid'))

In [ ]:
bidi_model.compile(optimizer = 'adam',
                   loss = 'binary_crossentropy',
                   metrics = ['accuracy'])
print(bidi_model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 32)           32000     
                                                                 
 bidirectional (Bidirection  (None, 256)               164864    
 al)                                                             
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 229889 (898.00 KB)
Trainable params: 229889 (898.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
cp = tf.keras.callbacks.ModelCheckpoint('MybestModelBIDI.h5',
                                        monitor='val_accuracy',
                                        save_weights=True,
                                        save_best_only = True,
                                        verbose=1)

In [ ]:
result = bidi_model.fit(train_sent_s,
                        y_train,
                        validation_data=(test_sent_s,y_test),
                        epochs = 20,
                        batch_size = 500,
                        callbacks = [cp])

Epoch 1/20
50/50 [==============================] - ETA: 0s - loss: 1.6475 - accuracy: 0.5914
Epoch 1: val_accuracy improved from -inf to 0.67400, saving model to MybestModelBIDI.h5
50/50 [==============================] - 9s 74ms/step - loss: 1.6475 - accuracy: 0.5914 - val_loss: 1.0811 - val_accuracy: 0.6740
Epoch 2/20


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


49/50 [============================>.] - ETA: 0s - loss: 0.7766 - accuracy: 0.7427
Epoch 2: val_accuracy improved from 0.67400 to 0.77832, saving model to MybestModelBIDI.h5
50/50 [==============================] - 2s 46ms/step - loss: 0.7719 - accuracy: 0.7441 - val_loss: 0.5779 - val_accuracy: 0.7783
Epoch 3/20
49/50 [============================>.] - ETA: 0s - loss: 0.5150 - accuracy: 0.7895
Epoch 3: val_accuracy improved from 0.77832 to 0.79280, saving model to MybestModelBIDI.h5
50/50 [==============================] - 2s 45ms/step - loss: 0.5150 - accuracy: 0.7892 - val_loss: 0.4777 - val_accuracy: 0.7928
Epoch 4/20
49/50 [============================>.] - ETA: 0s - loss: 0.4473 - accuracy: 0.8049
Epoch 4: val_accuracy improved from 0.79280 to 0.79552, saving model to MybestModelBIDI.h5
50/50 [==============================] - 2s 44ms/step - loss: 0.4464 - accuracy: 0.8052 - val_loss: 0.4517 - val_accuracy: 0.7955
Epoch 5/20
49/50 [============================>.] - ETA: 0s - loss

In [ ]:
new_model = tf.keras.models.load_model('MybestModelBIDI.h5')

In [ ]:
new_model.predict(new_reviews)

1/1 [==============================] - 0s 20ms/step


array([[0.9539706]], dtype=float32)